In [ ]:
import pandas as pd
import numpy as np

# 1. Cargar datos crudos
# Asumiendo que estás en la carpeta 'notebooks', el archivo está en '../data'
ruta_archivo = '../data/greatest_colombian_business.csv'
df_raw = pd.read_csv(ruta_archivo)

# 2. Filtrar Ecopetrol (El Outlier)
# Buscamos variaciones del nombre por si acaso, o usamos el NIT si es constante
sin_ecopetrol = df_raw[~df_raw['RAZÓN SOCIAL'].str.contains('ECOPETROL', case=False, na=False)].copy()

# 3. Limpieza de columnas numéricas (Quitar $ y ,)
cols_a_limpiar = ['INGRESOS OPERACIONALES', 'GANANCIA (PÉRDIDA)', 
                  'TOTAL ACTIVOS', 'TOTAL PASIVOS', 'TOTAL PATRIMONIO']

for col in cols_a_limpiar:
    # Solo limpiamos si es tipo texto (object)
    if sin_ecopetrol[col].dtype == 'O':
        # Reemplazar '$' y ',' por vacío
        sin_ecopetrol[col] = sin_ecopetrol[col].astype(str).str.replace('$', '', regex=False)
        sin_ecopetrol[col] = sin_ecopetrol[col].str.replace(',', '', regex=False)
        # Convertir a numérico (si falla pone NaN)
        sin_ecopetrol[col] = pd.to_numeric(sin_ecopetrol[col], errors='coerce')

# 4. Eliminar filas que hayan quedado con valores nulos en la variable objetivo
sin_ecopetrol = sin_ecopetrol.dropna(subset=['GANANCIA (PÉRDIDA)'])

print("✅ Datos cargados y limpios.")
print(f"Filas originales: {len(df_raw)}")
print(f"Filas limpias (sin Ecopetrol): {len(sin_ecopetrol)}")
print("Muestra de datos:")
display(sin_ecopetrol.head(3))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 1. Definir X e y
# Seleccionamos las variables predictoras.
# Descartamos NIT y RAZÓN SOCIAL porque son identificadores y no predicen patrones.
columnas_predictoras = [
    'INGRESOS OPERACIONALES', 
    'TOTAL ACTIVOS', 
    'TOTAL PASIVOS', 
    'TOTAL PATRIMONIO', 
    'MACROSECTOR', 
    'REGIÓN'
]

X = sin_ecopetrol[columnas_predictoras]
y = sin_ecopetrol['GANANCIA (PÉRDIDA)']

# 2. Dividir en Entrenamiento (80%) y Prueba (20%)
# random_state=42 asegura que si corres esto de nuevo, la división sea la misma.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Entrenando con {len(X_train)} empresas. Probando con {len(X_test)}.")

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error

# --- A. PREPROCESAMIENTO AUTOMÁTICO ---
# Definimos qué columnas son numéricas y cuáles categóricas
cols_num = ['INGRESOS OPERACIONALES', 'TOTAL ACTIVOS', 'TOTAL PASIVOS', 'TOTAL PATRIMONIO']
cols_cat = ['MACROSECTOR', 'REGIÓN']

# Creamos el transformador: 
# - A las numéricas las escala (StandardScaler ayuda a que el modelo converja mejor)
# - A las categóricas las vuelve dummies (0s y 1s)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), cols_num),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cols_cat)
    ])

# --- B. DEFINIR LOS COMPETIDORES ---
modelos = {
    "Regresión Lineal": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# --- C. ENTRENAR Y EVALUAR ---
resultados = {}

print("Iniciando entrenamiento...\n")

for nombre, modelo in modelos.items():
    # Creamos el Pipeline: 1. Preprocesamiento -> 2. Modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', modelo)])
    
    # Entrenamos con el 80% de datos
    pipeline.fit(X_train, y_train)
    
    # Predecimos con el 20% restante (datos que el modelo nunca vio)
    y_pred = pipeline.predict(X_test)
    
    # Evaluamos
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    resultados[nombre] = {"R2 Score": r2, "Error Absoluto Medio (MAE)": mae}
    print(f"✅ {nombre} procesado.")

# Ver tabla de resultados ordenada por mejor R2
df_resultados = pd.DataFrame(resultados).T
df_resultados.sort_values(by="R2 Score", ascending=False)

In [ ]:
import joblib
import os

# Aseguramos que la carpeta models exista
os.makedirs('../models', exist_ok=True)

print("Entrenando el modelo final con Random Forest...")

# 1. Creamos el Pipeline final con el ganador
# Usamos los mismos pasos: Preprocessor -> Random Forest
modelo_final = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# 2. Entrenamos con EL 100% DE LOS DATOS (X e y completos)
modelo_final.fit(X, y)

# Evaluar el modelo final sobre los mismos datos con los que se entrenó
# (OJO: Esto siempre dará un resultado más alto que la realidad)
y_pred_total = modelo_final.predict(X)

r2_total = r2_score(y, y_pred_total)
mae_total = mean_absolute_error(y, y_pred_total)

print(f"Métrica de ENTRENAMIENTO (Memoria): R2 = {r2_total:.2f}")
print(f"Métrica REAL estimada (Validación): R2 = 0.61")
# 3. Guardamos el archivo
ruta_modelo = '../models/modelo_ganancias_rf.pkl'
joblib.dump(modelo_final, ruta_modelo, compress=3)

print(f"✅ ¡Listo! Modelo guardado exitosamente en: {ruta_modelo}")
print("Ya puedes usar este archivo en tu interfaz de Streamlit.")
tamano_mb = os.path.getsize(ruta_modelo) / (1024 * 1024)
print(f"Nuevo tamaño del modelo: {tamano_mb:.2f} MB")

In [ ]:
# Obtener valores únicos ordenados alfabéticamente
sectores = sorted(sin_ecopetrol['MACROSECTOR'].unique().astype(str))
regiones = sorted(sin_ecopetrol['REGIÓN'].unique().astype(str))

print("--- COPIA Y PEGA ESTO EN TU DASHBOARD.PY ---")
print(f"lista_sectores = {sectores}")
print("\n")
print(f"lista_regiones = {regiones}")